<a href="https://colab.research.google.com/github/sugiyama404/ReinfoceLearningForTrading/blob/main/Ape_X_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import random
from google.colab import drive
import copy

from datetime import datetime
from matplotlib import pyplot as plt
import pickle

from tensorflow.keras.models import Sequential, clone_model
from tensorflow.keras.layers import Dense, ReLU, Input, Lambda
from tensorflow.keras.optimizers import Adam
from statistics import mean
import tensorflow.keras as keras
from tensorflow.keras.losses import Huber
from tensorflow.keras import backend as K

from sklearn.preprocessing import StandardScaler


mode = 'train'
name = 'apex'

drive.mount('/content/drive/')
nov_dir = 'Colab Notebooks/dataset/reinforcement_learning/'
nov_path = '/content/drive/My Drive/' + nov_dir + f'sp500_{mode}.csv'

exp_dir = 'Colab Notebooks/workspace/export/'
mdl_dir = '/content/drive/My Drive/' + exp_dir + 'models'
csv_path = '/content/drive/My Drive/' + exp_dir + f'csv_data/{name}_{mode}.csv'

df = pd.read_csv(nov_path)
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
class Environment:
    def __init__(self, df, initial_money=100000, mode = 'test'):

        self.df = df.dropna().reset_index()

        self.df_total_steps  = len(self.df)-1
        self.initial_money   = initial_money
        self.mode            = mode
        self.trade_time      = None
        self.trade_win       = None
        self.brfore_buy_cash = None
        self.action_space    = np.array([0, 1, 2]) # buy,hold,sell
        self.hold_a_position = None
        self.now_price       = None
        self.cash_in_hand    = None

        self.reset()
        
    def reset(self):

        self.trade_time      = 0
        self.trade_win       = 0
        self.brfore_buy_cash = 0
        self.end_step        = self.df_total_steps
        self.now_step        = 0
        self.hold_a_position = 0.0
        self.now_price       = self.df.loc[self.now_step, 'SP500']
        self.cash_in_hand    = self.initial_money

        return self._get_now_state()

    def step(self, action):

        prev_revenue = self._get_revenue()
        self.now_step += 1
        self.now_price = self.df.loc[self.now_step, 'SP500']
 
        done = (self.end_step == self.now_step)

        self._trade(action,done)
        cur_revenue = self._get_revenue()
 
        reward = cur_revenue - prev_revenue

        if self.mode == 'test':
            info = { 'cur_revenue' : cur_revenue , 'trade_time' : self.trade_time, 'trade_win' : self.trade_win }
        else:
            info = { 'cur_revenue' : cur_revenue }

        return self._get_now_state(), reward, done, info

    def _get_now_state(self):
        state = np.empty(3)
        state[0] = self.hold_a_position
        state[1] = self.now_price
        state[2] = self.cash_in_hand
        return state

    def _get_revenue(self): 
        return self.hold_a_position * self.now_price + self.cash_in_hand

    def _trade(self, action,lastorder = False):
        if lastorder:
            self.cash_in_hand += self.now_price * self.hold_a_position
            self.hold_a_position = 0
            if self.mode == 'test':
                self.trade_time += 1
                if self.cash_in_hand > self.brfore_buy_cash:
                    self.trade_win += 1
        else:
            if self.action_space[0] == action: # buy
                if self.hold_a_position == 0:
                    buy_flag = True
                    if self.mode == 'test':
                        self.brfore_buy_cash = copy.copy(self.cash_in_hand)
                    while buy_flag:
                        if self.cash_in_hand > self.now_price:
                            self.hold_a_position += 1
                            self.cash_in_hand -= self.now_price
                        else:
                            buy_flag = False
            if self.action_space[2] == action: # sell
                if self.hold_a_position != 0:
                    self.cash_in_hand += self.now_price * self.hold_a_position
                    self.hold_a_position = 0
                    if self.mode == 'test':
                        self.trade_time += 1
                        if self.cash_in_hand > self.brfore_buy_cash:
                            self.trade_win += 1

In [3]:
class Brain:
    def __init__(self):

        learning_rate = 0.00001
        neurons_per_layer = 24

        input = Input(shape=(3,))
        common = Dense(neurons_per_layer*2, activation='relu')(input)
        common = Dense(neurons_per_layer*4, activation='relu')(common)

        common = Dense(4, activation='linear')(common)
        output = Lambda(lambda a: K.expand_dims(a[:, 0], -1) + a[:, 1:] - 0.0*K.mean(a[:, 1:], keepdims=True),output_shape=(3,))(common)

        model = keras.Model(inputs=input, outputs=output)

        optimizer = Adam(learning_rate=learning_rate, clipvalue=40)
        model.compile(loss=Huber(), optimizer=optimizer)
        model.summary()
        self.model = model

In [4]:
class Learner:
    def __init__(self, model1, model2):

        self.model1 = model1
        self.model2 = model2
        self.gamma  = 0.99

    def learn(self, m_batch, s_flag):

        states, next_states, actions, rewards, done = m_batch['state'], m_batch['next_state'], m_batch['act'], m_batch['reward'], m_batch['done']

        next_act_values = self.model1.predict(next_states,s_flag)
        next_action =np.argmax(next_act_values)

        if s_flag == 11:
            q = self.model1.predict(states)  
            next_q = self.model2.predict(next_states)
            target = np.copy(q)

            target[:, actions] = rewards + (1 - done) * self.gamma*np.max(next_q, axis=1)
            self.model1.train_on_batch(states, target)
        else:
            q = self.model2.predict(states)  
            next_q = self.model1.predict(next_states)
            target = np.copy(q)

            target[:, actions] = rewards + (1 - done) * self.gamma*np.max(next_q, axis=1)
            self.model2.train_on_batch(states, target)

    def load(self, name, name2):
        self.model1.load_weights(name)
        self.model2.load_weights(name2)

    def save(self, name, name2):
        self.model1.save_weights(name)
        self.model2.save_weights(name2)

In [5]:
class Memory:
    def __init__(self, max_size=500, batch_size=32):

        self.cntr = 0
        self.size = 0
        self.max_size = max_size
        self.batch_size = batch_size
        self.states_memory = np.zeros([self.max_size, 3], dtype=np.float32)
        self.next_states_memory = np.zeros([self.max_size, 3], dtype=np.float32)
        self.acts_memory = np.zeros(self.max_size, dtype=np.uint8)
        self.rewards_memory = np.zeros(self.max_size, dtype=np.float32)
        self.done_memory = np.zeros(self.max_size, dtype=np.uint8)
        self.tderrors_memory = np.zeros(self.max_size, dtype=np.float32)

    def store_transition(self, state, act, reward, next_state, done):
        self.states_memory[self.cntr] = state
        self.next_states_memory[self.cntr] = next_state
        self.acts_memory[self.cntr] = act
        self.rewards_memory[self.cntr] = reward
        self.done_memory[self.cntr] = done
        self.cntr = (self.cntr+1) % self.max_size
        self.size = min(self.size+1, self.max_size)

    def random_sampling(self):
        mb_index = np.random.choice(self.size, self.batch_size, replace=False)
        key = ['state','next_state','act','reward','done']
        value = [self.states_memory[mb_index],self.next_states_memory[mb_index],
                 self.acts_memory[mb_index],self.rewards_memory[mb_index],
                 self.done_memory[mb_index]]
        dict1=dict(zip(key,value))
        return dict1

    def findall(self):
        return self.states_memory,self.next_states_memory,self.acts_memory,self.rewards_memory,self.done_memory,self.tderrors_memory

    def update_memory_tderror(self, val):
        self.tderrors_memory = val

In [6]:
class Agent:
    def __init__(self, model1, model2, memory, learner, epsilon, batch_size=32):
        self.model1 = model1
        self.model2 = model2
        self.memory = memory
        self.learner = learner
        self.gamma = 0.99
        self.epsilon = epsilon
        self.batch_size = batch_size

    def update_replay_memory(self, state, action, reward, next_state, done):
        self.memory.store_transition(state, action, reward, next_state, done)

    def act(self, state, s_flag=12):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(3)
        act_values = self._predict(state,s_flag)
        return np.argmax(act_values)

    def _predict(self, state, s_flag = 12):
        values = None
        q1 = self.model1.predict(state)
        q2 = self.model2.predict(state)
        if s_flag == 12:
            values = q1 + q2
        elif s_flag == 11:
            values = q1 + q1
        else:
            values = q2 + q2
        return values

    def replay(self):
        m_batch = self.memory.random_sampling()
        return m_batch

    def pioritized_experience_replay(self):

        sum_ab_tderror = self._absolute_tderror()
        td_list = np.random.uniform(0, sum_ab_tderror, self.batch_size)
        td_list = np.sort(td_list)

        num_np = np.array([], dtype=np.int16)
        i, sum_ab_tderror_tmp = 0, 0
        states, next_states, actions, rewards, done, tderror = self.memory.findall()
        for item in td_list:
            while sum_ab_tderror_tmp < item:
                sum_ab_tderror_tmp += abs(tderror[i]) + 0.0001
                i += 1
            num_np = np.append(num_np, i)

        key = ['state','next_state','act','reward','done']
        value = [states[num_np],next_states[num_np],
                 actions[num_np],rewards[num_np],
                 done[num_np]]
        dict1=dict(zip(key,value))
        return dict1

    def tderror(self):
        states, next_states, acts, rewards, done, tderrors = self.memory.findall()
        
        n2_rewards = rewards[2:-1]
        next_rewards = rewards[1:-2]
        n3_action = acts[3:]
        n3_next_states = next_states[3:]

        states = states[:-3]
        next_states = next_states[:-3]
        acts = acts[:-3]
        rewards = rewards[:-3]
        done = done[:-3]

        #self.memory.cntr = (self.memory.cntr - 3) if (self.memory.cntr - 3) > 0 else (self.memory.cntr + max_size - 4)
        #self.memory.size = (self.memory.size - 3) if (self.memory.size - 3) > 0 else (self.memory.size + max_size - 4)

        self.memory.cntr = self.memory.cntr - 3
        self.memory.size = self.memory.size - 3

        next_action = np.argmax(self._predict(n3_next_states), axis = 1)
        next_action = next_action.reshape(-1)

        acts = acts.reshape(-1)

        acts_one = np.eye(3)[acts]
        next_action_one = np.eye(3)[next_action]
        n3_q = np.max(self._predict(n3_next_states) * next_action_one, axis = 1)

        target = rewards + self.gamma * next_rewards + (self.gamma ** 2) * n2_rewards + (self.gamma ** 3) * n3_q
        tderror = target - np.max(self._predict(states) * acts_one, axis = 1)
        self.memory.update_memory_tderror(tderror)

    def _absolute_tderror(self):
        absolute_tderror = 0
        tderror = self.memory.tderrors_memory
        for i in range(0, (len(tderror)-1)):
            absolute_tderror += abs(tderror[i]) + 0.0001
        return absolute_tderror

    def integration(self):
        self.model1 = clone_model(self.learner.model1)
        self.model2 = clone_model(self.learner.model2)

In [7]:
class Main:
    def __init__(self, env, agent, learner, mdl_dir, name, episodes_times = 200, mode = 'test'):
        self.env            = env
        self.agent          = agent
        self.learner        = learner
        self.mdl_dir        = mdl_dir
        self.scaler         = self._standard_scaler(self.env)
        self.episodes_times = episodes_times
        self.mode           = mode
        self.name           = name

        if self.mode == 'test':
            self._load()
            self.agent.epsilon = 0.01

            with open(csv_path, 'w') as f:
                row = 'FixedProfit,TradeTimes,TradeWin'
                print(row, file=f)
        else:
            with open(csv_path, 'w') as f:
                row = 'FixedProfit'
                print(row, file=f)

    def play_game(self):

        #total_reward = [0]
        total_reward = [1000000]
        self.agent.integration()

        for episode in range(self.episodes_times):
            state = self.env.reset()
            state = self.scaler.transform([state])
            done  = False
            start_time = datetime.now()
        
            while not done:
                s_flag = 12
                action = self.agent.act(state)
                next_state, reward, done, info = self.env.step(action)
                next_state = self.scaler.transform([next_state])

                if self.mode == 'train':
                    self.agent.update_replay_memory(state, action, reward, next_state, done)
              
            play_time = datetime.now() - start_time
            if self.mode == 'test':
                print("Episode: {}/{} RapTime: {} FixedProfit: {:.0f} TradeTimes: {} TradeWin: {}".format(episode + 1, episodes_times, play_time, info['cur_revenue'], info['trade_time'], info['trade_win']))
                with open(csv_path, 'a') as f:
                    row = str(info['cur_revenue']) + ',' + str(info['trade_time']) + ',' + str(info['trade_win'])
                    print(row, file=f)
            else:
                self.agent.tderror()
                if mean(total_reward) < 1050000:
                    m_batch = self.agent.replay()
                else:
                    m_batch = self.agent.pioritized_experience_replay()
                s_flag = 11 if np.random.random() <= 0.5 else 22
                learner.learn(m_batch, s_flag)
 
                self.agent.tderror()
                total_reward.append(info['cur_revenue'])
                agent.integration()
                print("Episode: {}/{} RapTime: {} FixedProfit: {:.0f}".format(episode + 1, episodes_times, play_time, info['cur_revenue']))
                with open(csv_path, 'a') as f:
                    row = str(info['cur_revenue'])
                    print(row, file=f)
    
            state = next_state

        if self.mode == 'train':
            self._save()

    def _standard_scaler(self, env):
        states = []
        for _ in range(env.df_total_steps):
            action = np.random.choice(env.action_space)
            state, reward, done, info = env.step(action)
            states.append(state)
            if done:
                break
        
        scaler = StandardScaler()
        scaler.fit(states)
        return scaler

    def _load(self):
        with open('{}/{}.pkl'.format(self.mdl_dir, self.name), 'rb') as f:
            self.scaler = pickle.load(f)
        self.learner.load('{}/{}_1.h5'.format(self.mdl_dir, self.name), '{}/{}_2.h5'.format(self.mdl_dir, self.name))


    def _save(self):
        self.learner.save('{}/{}_1.h5'.format(self.mdl_dir, self.name), '{}/{}_2.h5'.format(self.mdl_dir, self.name))
        with open('{}/{}.pkl'.format(self.mdl_dir, self.name), 'wb') as f:
            pickle.dump(self.scaler, f)

In [8]:
initial_money=1000000
episodes_times = 200
batch_size = 128
max_size = 500

env = Environment(df, initial_money=initial_money, mode = mode)

brain1 = Brain()
model1 = brain1.model
brain2 = Brain()
model2 = brain2.model

learner = Learner(model1, model2)
memory = Memory(max_size, batch_size)
epsilon_list = [0.4]


epsilon = epsilon_list[0]
model1_cp = clone_model(model1)
model2_cp = clone_model(model2)
agent = Agent(model1_cp, model2_cp, memory, learner, epsilon, batch_size)
main = Main(env, agent, learner, mdl_dir, name, episodes_times, mode)
main.play_game()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense (Dense)                (None, 48)                192       
_________________________________________________________________
dense_1 (Dense)              (None, 96)                4704      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 388       
_________________________________________________________________
lambda (Lambda)              (None, 3)                 0         
Total params: 5,284
Trainable params: 5,284
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape 